## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/Weather_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Dikson,RU,73.5069,80.5464,-15.92,100,69,3.94,broken clouds
1,1,Butaritari,KI,3.0707,172.7902,83.07,74,85,13.96,overcast clouds
2,2,Felanitx,ES,39.4696,3.1483,51.01,80,96,7.07,overcast clouds
3,3,San Luis,AR,-33.2950,-66.3356,76.24,49,56,9.75,broken clouds
4,4,Biak,ID,-0.9131,122.8766,80.17,79,100,0.43,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
#ask the customer to add a minimum and max temp value
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 0
What is the maximum temperature you would like for your trip? 20


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]<=max_temp) &
                                      (city_data_df["Max Temp"]>= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

#country  has 6 NA because there are 6 cities with country code NA

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
16,Lososina,RU,19.17,clear sky,48.9975,140.3106,
21,Taksimo,RU,0.97,overcast clouds,56.3444,114.8794,
23,Saint Anthony,US,7.56,clear sky,45.0205,-93.2180,
25,Qaanaaq,GL,19.60,overcast clouds,77.4840,-69.3632,
45,Timmins,CA,10.22,light snow,48.4669,-81.3331,
46,Zeya,RU,4.75,clear sky,53.7500,127.2667,
77,Nenjiang,CN,9.45,scattered clouds,49.1833,125.2167,
105,Arman,RU,9.79,overcast clouds,59.7000,150.1667,
106,Yankton,US,17.85,clear sky,42.8711,-97.3973,
120,Tuktoyaktuk,CA,12.20,broken clouds,69.4541,-133.0374,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np

hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df.head(15)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
16,Lososina,RU,19.17,clear sky,48.9975,140.3106,"Хостел ""Пилигрим"""
21,Taksimo,RU,0.97,overcast clouds,56.3444,114.8794,"Gostinitsa ""Domashniy Ochag"""
23,Saint Anthony,US,7.56,clear sky,45.0205,-93.2180,"Holiday Inn Express Roseville-St. Paul, an IHG..."
25,Qaanaaq,GL,19.60,overcast clouds,77.4840,-69.3632,Qaanaaq Hotel
45,Timmins,CA,10.22,light snow,48.4669,-81.3331,The Senator Hotel & Conference Centre
46,Zeya,RU,4.75,clear sky,53.7500,127.2667,Kapital Inn
77,Nenjiang,CN,9.45,scattered clouds,49.1833,125.2167,Nenjiang Apartment
106,Yankton,US,17.85,clear sky,42.8711,-97.3973,"Holiday Inn Express & Suites Yankton, an IHG H..."
120,Tuktoyaktuk,CA,12.20,broken clouds,69.4541,-133.0374,End of The Road Ltd.
122,Segezha,RU,9.97,scattered clouds,63.7415,34.3222,Sampo Hotel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/Vacation_search.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))